# Project 4
## Students:
 > Steven Koprowicz, 
 > Matthew Walters 

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import re

In [2]:
print(tf.__version__)# you may want to upgrade to 2.10.0 

2.11.0


### Please Use Markdown
> for markdown, see here: https://www.ibm.com/docs/en/watson-studio-local/1.2.3?topic=notebooks-markdown-jupyter-cheatsheet

In [3]:
class TransformerModel():
    def __init__(self, vocab_size, embed_dim=256, num_heads=2, num_blocks=1, ff_dim=256, maxlen=64, rate=0.1):
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_blocks = num_blocks
        self.ff_dim = ff_dim
        self.maxlen = maxlen
        self.rate = rate

    def TransformerBlock(self, inputs):
        #create the transformer block as discribed in the writeup, 
        # use the Keras functional API (https://keras.io/guides/functional_api/)
        # Use the rate variable for the dropout layers
        
        # MultiHeadAttention layer, 
        # specifiy 'use_causal_mask=True' (https://keras.io/api/layers/attention_layers/multi_head_attention/)
        mha = layers.MultiHeadAttention(use_causal_mask=True)
        
        # drop-out layer
        
        # sum inputs and the output of this drop-out layer
        
        # LayerNormalization layer, 
        # specifiy 'epsilon=1e-6' (https://keras.io/api/layers/normalization_layers/layer_normalization/)
        ln = layers.LayerNormalization(epsilon=1e-6)
        
        # dense
        
        # dense
        
        # drop-out layer
        
        # sum the output of the first Layer Normalization layer and this drop-out layer
        
        # LayerNormalization again
        
        #output
    
    def EmbeddingLayer(self, inputs):
        #create the embedding layer
        
        #create (1) an embedding for the tokens and (2) an embedding for the positions
        #you can use https://keras.io/api/layers/core_layers/embedding/ Embedding class
        #you can use tf.range to encode positions
        #add (1) and (2) and return the layer
    
    def create_model(self):
        #combine the EmbeddingLayer and num_blocks TransformerBlocks to create the model, 
        # use the Keras functional API (https://keras.io/guides/functional_api/)
        #use the SparseCategoricalCrossentropy loss function 
        # (https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class)

IndentationError: expected an indented block (41036079.py, line 24)

## Task 1

In [1]:
class TransformerModel():
    def __init__(self, vocab_size, embed_dim=256, num_heads=2, num_blocks=1, ff_dim=256, maxlen=64, rate=0.1):
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_blocks = num_blocks
        self.ff_dim = ff_dim
        self.maxlen = maxlen
        self.rate = rate

    def TransformerBlock(self, inputs):
        #create the transformer block as discribed in the writeup, 
        # use the Keras functional API (https://keras.io/guides/functional_api/)
        # Use the rate variable for the dropout layers
        
        # MultiHeadAttention layer, 
        # specifiy 'use_causal_mask=True' (https://keras.io/api/layers/attention_layers/multi_head_attention/)
        mha = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embed_dim, use_causal_mask=True)
         
        # drop-out layer
        first_dropout = layers.Dropout(self.rate)(mha)
        
        # sum inputs and the output of this drop-out layer
        first_sum = layers.Add()([inputs, first_dropout])

        # LayerNormalization layer, 
        # specifiy 'epsilon=1e-6' (https://keras.io/api/layers/normalization_layers/layer_normalization/)
        first_ln = layers.LayerNormalization(epsilon=1e-6)(first_sum)
        
        # dense
        first_dense = layers.Dense(self.ff_dim, activation="relu")(first_ln)

        # dense
        second_dense = layers.Dense(self.ff_dim, activation="relu")(first_dense)

        # drop-out layer
        second_dropout = layers.Dropout(self.rate)(second_dense)

        # sum the output of the first Layer Normalization layer and this drop-out layer
        second_sum = layers.Add()([first_ln, second_dropout])

        # LayerNormalization again
        second_ln = layers.LayerNormalization(epsilon=1e-6)(first_sum)

        output_dense = layers.Dense(self.ff_dim, activation="relu")(second_ln)

        return output_dense
    
    def EmbeddingLayer(self, inputs):
        #create the embedding layer
        
        #create (1) an embedding for the tokens and (2) an embedding for the positions
        #you can use https://keras.io/api/layers/core_layers/embedding/ Embedding class
        #you can use tf.range to encode positions
        #add (1) and (2) and return the layer
    
    def create_model(self):
        #combine the EmbeddingLayer and num_blocks TransformerBlocks to create the model, 
        # use the Keras functional API (https://keras.io/guides/functional_api/)
        #use the SparseCategoricalCrossentropy loss function 
        # (https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class)

IndentationError: expected an indented block after function definition on line 40 (2862044581.py, line 48)

## Task 2

In [ ]:
class DataSet():
    def __init__(self, filename, len):
        #load the text from the file
        pass
        

    def prep_text(self):
        #remove all punctuation, set to lowercase, remove duplicate spaces and other whitespace (keep newlines)
        pass
        
        
    def tokenize_text(self):
        #seperate into words, create a vocab and convert the text to a list of numbers using the vocab such that each unique word is represented by its own number number
        pass
        

    def create_dataset(self):
        #split the tokenized data into sequences of length len, return the sequences and vocab
        pass

## Task 3

In [ ]:
class GenerateText():
    def __init__(self, model, vocab):
        pass

    
    def generate_text(self, start_string, num_generate=100):
        #generate text using the model and vocab, start with the start_string and generate num_generate words
        pass

## Task 4: Model Traning and Testing

In [ ]:
#Train the model while periodically generating text toshow progress
def train_model(model, vocab, x, y, epochs=50):
    
    return model


# Report

## Introduction

## Results

## Conclusion

## How to Run Code

Please include any special libraries and list your tf version here.